In [57]:
import pickle
import os
from datetime import datetime 
import pandas as pd
import numpy as np

In [89]:
model_path = r"C:\DS\Seoul-Bike-Sharing-Demand-Prediction-main\Models\xgboost_regressor_r2_0_928_v1.pkl"
model = pickle.load(open(model_path,"rb"))
 

In [90]:
model.predict([[ 1.07807977,  0.22192612,  0.77468297,  1.82005277, -1.26200749,
       -0.25194272, -0.12916836, -0.17535463, -0.22754885, -5.47803337,
        0.62948994,  1.92199598,  1.2340516 ,  1.74132735, -0.58152447,
       -0.57866842, -0.24538093, -0.24242567, -0.25819889, -0.24766057,
       -0.24242567, -0.24439892]])

array([403.82867], dtype=float32)

 ## Load SC

In [91]:
sc_dump_path = r"C:\DS\Seoul-Bike-Sharing-Demand-Prediction-main\Models\sc.pkl"

sc = pickle.load(open(sc_dump_path , "rb"))

In [92]:
sc

StandardScaler()

## User Input

In [93]:
date = "4/1/24"
hour = 14
temperature = 34
humidity = 55 
wind_speed = 67
visibility = 100
solar_radiation = 0.4 
rainfall = 0.7
functioning_day = "Yes"
snowfall = 0.2
seasons = "Winter"
holiday = "No Holiday"

## Converting User Data into Model Consumable format

In [94]:
holiday_dic = {"No Holiday":0,"Holiday":1}
finctioning_day= {"No" : 0 , "Yes" : 1}

def get_string_to_datetime(date):
    dt = datetime.strptime(date,"%d/%m/%y")
    return {"day" : dt.day,"month" : dt.month , "year" : dt.year ,"week_day":dt.strftime("%A")}

str_to_date = get_string_to_datetime(date)
str_to_date

{'day': 4, 'month': 1, 'year': 2024, 'week_day': 'Thursday'}

In [95]:
u_input_list = [hour, temperature, humidity, wind_speed, visibility, solar_radiation, rainfall, snowfall,
                holiday_dic[holiday],finctioning_day[functioning_day],str_to_date["day"],
                str_to_date["month"], str_to_date["year"]]



features_name = ["Hour",'Temperature(°C)', 'Humidity(%)', 'Wind speed (m/s)', 'Visibility (10m)', 'Solar Radiation (MJ/m2)',
       'Rainfall(mm)', 'Snowfall (cm)', 'Holiday', 'Functioning Day', 'Day','Month', 'Year']


df_u_input = pd.DataFrame([u_input_list],columns=features_name)
df_u_input

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,Day,Month,Year
0,14,34,55,67,100,0.4,0.7,0.2,0,1,4,1,2024


In [96]:
def season_to_df(seasons):
    seasons_cols = ['Spring', 'Summer', 'Winter']
    seaosns_data = np.zeros((1,len(seasons_cols)) ,dtype = "int")

    df_seasons = pd.DataFrame(seaosns_data,columns=seasons_cols)
    if seasons in seasons_cols :
        df_seasons[seasons]=1
        return df_seasons

df_seasons = season_to_df(seasons)
df_seasons

,Spring,Summer,Winter
0,0,0,1


In [97]:
str_to_date["week_day"]

'Thursday'

In [98]:
def days_df(week_day) : 
    days_names = ['Monday', 'Saturday','Sunday', 'Thursday', 'Tuesday', 'Wednesday']
    days_name_data = np.zeros((1,len(days_names)) , dtype = "int")

    df_days = pd.DataFrame(days_name_data,columns=days_names)

    if week_day in days_names :
        df_days[week_day] = 1
    return df_days 

df_days = days_df(str_to_date["week_day"])
df_days

,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,0,0,0,1,0,0


In [99]:
df_for_pred = pd.concat([df_u_input, df_seasons ,df_days],axis=1)
df_for_pred

,Hour,Temperature(°C),Humidity(%),Wind speed (m/s),Visibility (10m),Solar Radiation (MJ/m2),Rainfall(mm),Snowfall (cm),Holiday,Functioning Day,...,Year,Spring,Summer,Winter,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
0,14,34,55,67,100,0.4,0.7,0.2,0,1,...,2024,0,0,1,0,0,0,1,0,0


In [100]:
sc_data_for_pred = sc.transform(df_for_pred)
sc_data_for_pred

array([[ 0.3562424 ,  1.76076456, -0.15409119, 63.11301355, -2.21005725,
        -0.19410378,  0.46358031,  0.29083855, -0.22754885,  0.18254726,
         0.3698698 , -0.4096857 ,  1.24013023, -0.57427456, -0.58152447,
         1.72810536, -0.24538093, -0.24242567, -0.25819889,  4.0377845 ,
        -0.24242567, -0.24439892]])

## Prediction in Deployment

In [103]:
# model.predict(sc_data_for_pred)

print(f"Rented Bike Demand on date : {date}, and Time : {hour} is : {round(model.predict(sc_data_for_pred).tolist()[0])}")

Rented Bike Demand on date : 4/1/24, and Time : 14 is : 325
